In [1]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
# from sklearnex import patch_sklearn
# patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import os

max_length=128
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-04-15 20:12:14.857502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 20:12:14.857607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 20:12:14.983773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/emotion-dataset/train_file.json
/kaggle/input/emotion-dataset/val_file.json


In [2]:
import wandb

# Set your wandb API key
wandb_api_key = "e62fc492915628e64fcac9c082089ffed84dc72d"

# Log in to wandb using the API key
wandb.login(key=wandb_api_key)

# Initialize wandb
wandb.init(project="nlp4-1")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240415_201224-ju20snob
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rural-shadow-41
wandb: ⭐️ View project at https://wandb.ai/karan912/nlp4-1
wandb: 🚀 View run at https://wandb.ai/karan912/nlp4-1/runs/ju20snob/workspace


In [3]:
PATH='/kaggle/input/emotion-dataset/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_encoder = LabelEncoder()

In [4]:
# read json
def dataframe(file):
    with open(PATH+file) as f:
        data = json.load(f)
        # to pandas
        df = pd.DataFrame(data)
    return df


In [5]:
def data_creator2(df):
    sentence_len = []
    text1=[]
    emotions=[]
    text2=[]
    for j in range(len(df)):
        uterances=df.iloc[j]['utterances']
        speakers=df.iloc[j]['speakers']
        emotion=df.iloc[j]['emotions']
        emote=[]
        length=0
        sentence=""
        
        for i in range(len(speakers)):
            speaker=speakers[i]
            uterance=uterances[i]
            text1.append(sentence)
            sentence+=speaker+': '+uterance+' '
            text2.append(speakers[i]+': '+uterances[i]+' ')
            emotions.append(emotion[i])
        
#         for i in range(len(speakers)-1):
#             speaker=speakers[i]
#             uterance=uterances[i]
            
#             sentence+=speaker+': '+uterance+' '
            
#         text2.append(speakers[-1]+': '+uterances[-1]+' ')
#         text1.append(sentence)
#         emotions.append(emotion[-1])
    print(len(text1),len(text2),len(emotions))
    return {"context":text1,"text":text2,"emotions":emotions}



def data_creator(df):
    sentence_len = []
    text1=[]
    emotions=[]
    text2=[]
    for j in range(len(df)):
        uterances=df.iloc[j]['utterances']
        speakers=df.iloc[j]['speakers']
        emotion=df.iloc[j]['emotions']
        emote=[]
        length=0
        sentence=""
        
#         for i in range(len(speakers)):
#             speaker=speakers[i]
#             uterance=uterances[i]
#             text1.append(sentence)
#             sentence+=speaker+': '+uterance+' '
#             text2.append(speakers[-1]+': '+uterances[-1]+' ')
#             emotions.append(emotion[-1])
        
        for i in range(len(speakers)-1):
            speaker=speakers[i]
            uterance=uterances[i]
            
            sentence+=speaker+': '+uterance+' '
            
        text2.append(speakers[-1]+': '+uterances[-1]+' ')
        text1.append(sentence)
        emotions.append(emotion[-1])
    print(len(text1),len(text2),len(emotions))
    return {"context":text1,"text":text2,"emotions":emotions}
        # print(length)
        # break


In [6]:
# config =AutoConfig.from_pretrained(model_id)
# config.update({"id2label": label_to_id})
# tokenizer = RobertaTokenizerFast.from_pretrained(model_id)
# model = RobertaForSequenceClassification.from_pretrained(model_id,config=config)
# train_file="MELD_train_efr.json"
# val_file="MELD_dev_efr.json"
# test_file="MELD_test_efr.json"
train_file="train_file.json"
val_file="val_file.json"


In [7]:
label_encoding ={
    'surprise':0,
    'sadness':1,
    'anger':2,
    'fear':3,
    'disgust':4,
    'joy':5,
    'neutral':6
}


In [8]:
from tqdm.notebook import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from torch import nn
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
def label_encode(x):
    return label_encoding[x]

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data=data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):   
        labels=self.data.iloc[idx,2]
        sentence=self.data.iloc[idx,0]+"</s></s>"+ self.data.iloc[idx,1]
        encoded_pair = self.tokenizer(sentence,
                                      max_length=128,
                                      add_special_tokens=True,
                                      padding='max_length', 
                                      truncation=True, 
                                      return_tensors='pt')
        input_ids = encoded_pair['input_ids'].squeeze(0)
        attention_mask = encoded_pair['attention_mask'].squeeze(0)
        return input_ids,attention_mask,torch.tensor(labels)

In [11]:
from torch.nn.functional import softmax
import gc
gc.collect()
torch.cuda.empty_cache()
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop"
def train_epoch(model,optimizer,epoch):
    model.train()
    total_loss = 0
    progress=tqdm(train_dataloader, desc=f"Epoch:{epoch}",total=len(train_dataloader), leave=False)
    for batch in progress:
        optimizer.zero_grad()
        targets = batch[2].to(device)
        input_ids=batch[0].to(device)
        attention_mask=batch[1].to(device)
        outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=targets)
        probs = softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1)
#         print(pred,targets)
#         logits = pre
        loss=outputs.loss

#         loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        del input_ids
        del attention_mask
        del targets
        gc.collect()
        torch.cuda.empty_cache()
        progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})
    wandb.log({'epoch':epoch,'train_loss':total_loss / len(train_dataloader)})
    print(f"Epoch {epoch}, Average Loss: {total_loss / len(train_dataloader)}")
    return total_loss / len(train_dataloader)

def evaluate(model):
    with torch.no_grad():
        model.eval()
        losses = 0
        progress=tqdm(val_dataloader, desc=f"Epoch:{epoch}",total=len(val_dataloader), leave=False)
        for batch in progress:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            outputs=model(input_ids,attention_mask=attention_mask,labels=labels)
            probs = softmax(outputs.logits, dim=1)
            pred = torch.argmax(probs, dim=1)
            loss=outputs.loss.item()
            losses+=loss
            del input_ids
            del attention_mask
            del labels
            gc.collect()
            torch.cuda.empty_cache()
            progress.set_postfix({'val_loss': f'{loss/len(batch):.3f} len={len(val_dataloader)}'})
        wandb.log({'epoch':epoch,'val_loss':losses / len(val_dataloader)})
        x=losses/len(val_dataloader)
        return x

In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
def getloader2(batch_size,file):
    train_dataframe=dataframe(file)
    train_data1=data_creator2(train_dataframe)
    train_data=pd.DataFrame(train_data1)
    print(train_data)
    train_data.to_csv(file.split(".")[0]+".csv")
    train_data['emotions']=train_data['emotions'].apply(label_encode)
    train_dataset = Dataset(train_data,tokenizer)
    train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)
    return train_dataloader

def getloader(batch_size,file):
    train_dataframe=dataframe(file)
    train_data1=data_creator(train_dataframe)
    train_data=pd.DataFrame(train_data1)
    train_data.to_csv(file.split(".")[0]+".csv")
    train_data['emotions']=train_data['emotions'].apply(label_encode)
    train_dataset = Dataset(train_data,tokenizer)
    train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)
    return train_dataloader

def testingloader(file,model):
    dataloader=getloader2(batch_size,file)
    preds=[]
    actual=[]
    losses = 0
    with torch.no_grad():
        model.eval()
        progress=tqdm(dataloader, desc=f"Testing",total=len(dataloader), leave=False)
        for batch in progress:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            targets =batch[2].to(device)
            outputs=model(input_ids,attention_mask=attention_mask,labels=targets)
            probs = softmax(outputs.logits, dim=1)
            pred = torch.argmax(probs, dim=1)
            loss=outputs.loss.item()
            losses+=loss
            preds.extend(pred)
            actual.extend(targets)
            del input_ids
            del attention_mask
            del targets
            gc.collect()
            torch.cuda.empty_cache()
            progress.set_postfix({'test_loss': f'{loss/len(batch):.3f} len={len(dataloader)}'})
            wandb.log({'test_loss':losses / len(dataloader)})
    print(f"test loss: {losses / len(dataloader)}")
    return preds,actual
    

In [14]:
batch_size=128
from torch.utils.data import DataLoader
train_dataloader=getloader2(batch_size,train_file)
val_dataloader=getloader2(batch_size,val_file)

58957 58957 58957
                                                 context  \
0                                                          
1             Phoebe: You-you
you had sex with Ursula?!    
2      Phoebe: You-you
you had sex with Ursula?! Eric...   
3      Phoebe: You-you
you had sex with Ursula?! Eric...   
4      Phoebe: You-you
you had sex with Ursula?! Eric...   
...                                                  ...   
58952                                                      
58953  Rachel: Can you believe what a jerk Ross was b...   
58954  Rachel: Can you believe what a jerk Ross was b...   
58955  Rachel: Can you believe what a jerk Ross was b...   
58956  Rachel: Can you believe what a jerk Ross was b...   

                                                    text  emotions  
0             Phoebe: You-you
you had sex with Ursula?!   surprise  
1      Eric: Uh, a little bit. She-she-she walked in ...      fear  
2      Phoebe: You didn't notice she was wearing 

In [15]:

# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=7)
# # model.classifier = nn.Sequential(
# #     nn.Linear(model.config.hidden_size, 7),
# #     nn.Softmax()  # Output float between 0 and 1
# # )
# model = model.to(device)
# model
# optimizer = AdamW(model.parameters(), lr=5e-5)
# loss_fn = torch.nn.CrossEntropyLoss()
# NUM_EPOCHS=10
# for epoch in range(1, NUM_EPOCHS+1):
#     train_loss = train_epoch(model, optimizer,epoch)
#     val_loss = evaluate(model)
#     torch.save(model, f"model1A_epoch_{epoch}.pth")
#     wandb.log({"Epoch": epoch, "Train loss": train_loss, "Val loss": val_loss})
#     print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:1:   0%|          | 0/461 [00:00<?, ?it/s]

Epoch 1, Average Loss: 0.9838087945336116


Epoch:1:   0%|          | 0/57 [00:00<?, ?it/s]

...

Epoch: 9, Train loss: 0.235, Val loss: 0.226


Epoch:10:   0%|          | 0/461 [00:00<?, ?it/s]

Epoch 10, Average Loss: 0.2335536952762826


Epoch:10:   0%|          | 0/57 [00:00<?, ?it/s]

Epoch: 10, Train loss: 0.234, Val loss: 0.218


In [16]:
# torch.save(model,"/kaggle/working/M1A_final.pth")

In [17]:
batch_size=128
loaded_model=torch.load("/kaggle/working/M1A_final.pth")
loaded_model.to(device)

In [18]:
test_file="val_file.json"
preds,actuals=testingloader(test_file,loaded_model)

7293 7293 7293
                                                context  \
0                                                         
1     Chandler: Hey! Hold on a minute, hold on a sec...   
2     Chandler: Hey! Hold on a minute, hold on a sec...   
3     Chandler: Hey! Hold on a minute, hold on a sec...   
4     Chandler: Hey! Hold on a minute, hold on a sec...   
...                                                 ...   
7288  Monica: Something went wrong with Underdog, an...   
7289  Monica: Something went wrong with Underdog, an...   
7290  Monica: Something went wrong with Underdog, an...   
7291  Monica: Something went wrong with Underdog, an...   
7292  Monica: Something went wrong with Underdog, an...   

                                                   text  emotions  
0     Chandler: Hey! Hold on a minute, hold on a sec...     anger  
1             Joey: I'd really prefer a mountain bike.    neutral  
2     Chandler: Janice's birthday is coming up, I wa...   neutral  
3   

Testing:   0%|          | 0/57 [00:00<?, ?it/s]

test loss: 0.21845453637733794


In [19]:
preds2=[]

for i in preds:
        preds2.append(i.item())
actuals2=[]
for i in actuals:
        actuals2.append(i.item())
        
# print(len(preds2),len(actuals2))
# preds=torch.stack(preds.)
# actuals=torch.stack(actuals)
# print(preds.shape,actuals.shape)

In [20]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [21]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
#metrics
#preds,actuals
f1_sccore = f1_score(actuals2, preds2, average='weighted')
f1_macro = f1_score(actuals2, preds2, average='macro')
print(f1_macro,f1_sccore)
print(classification_report(actuals2, preds2))

0.8903389945726712 0.9086820215441184
              precision    recall  f1-score   support

           0       0.94      0.87      0.91      1008
           1       0.94      0.84      0.88       558
           2       0.94      0.84      0.89       788
           3       0.87      0.84      0.86       265
           4       0.95      0.80      0.87       215
           5       0.89      0.91      0.90      1259
           6       0.90      0.96      0.93      3200

    accuracy                           0.91      7293
   macro avg       0.92      0.87      0.89      7293
weighted avg       0.91      0.91      0.91      7293

